In [ ]:
# Cella 1: Import e configurazione
import os
import random
import time
import math
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
from torchvision.models.detection.ssd import SSDClassificationHead
from torchvision.models.detection import _utils
from torchvision.ops import box_iou
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.model_selection import KFold
import wandb
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Riproducibilità
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Configurazione
PROJECT = "pothole-detector-NatureSR"
ENTITY = "pothole-detector"
NUM_FOLDS = 4
IOU_THRESHOLD = 0.7
CONF_THRESHOLD = 0.25  # CORRETTO: aumentato da 0.01 a 0.25
INPUT_SIZE = 320

# Parametri training
CONFIG = {
    "batch_size": 16,
    "learning_rate": 0.001,
    "weight_decay": 0.0001,
    "momentum": 0.9,
    "epochs": 100,
    "patience": 10,
}

CLASS_NAMES = ["pothole", "crack", "manhole"]

# Login wandb
wandb.login(key="<API_KEY>")
print("✓ Setup completato")

In [ ]:
# Cella 2: Creazione modello
def create_model(num_classes=4, size=320):
    """Crea SSD320 con MobileNetV3 backbone"""
    model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1)
    
    # Modifica testa classificazione
    in_channels = _utils.retrieve_out_channels(model.backbone, (size, size))
    num_anchors = model.anchor_generator.num_anchors_per_location()
    
    model.head.classification_head = SSDClassificationHead(
        in_channels=in_channels,
        num_anchors=num_anchors,
        num_classes=num_classes
    )
    
    model.transform.min_size = (size,)
    model.transform.max_size = size
    
    return model

# Test modello
model = create_model()
total_params = sum(p.numel() for p in model.parameters())
print(f"✓ Modello creato: {total_params:,} parametri")

In [ ]:
# Cella 3: Dataset
class PotholeDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_files, input_size=320):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_files = image_files
        self.input_size = input_size
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        original_w, original_h = image.size

        label_name = img_name.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(self.label_dir, label_name)

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 9:
                        class_id = int(parts[0])
                        coords = list(map(float, parts[1:]))

                        xs = coords[0::2]
                        ys = coords[1::2]

                        x_min = min(xs) * original_w
                        y_min = min(ys) * original_h
                        x_max = max(xs) * original_w
                        y_max = max(ys) * original_h

                        w = x_max - x_min
                        h = y_max - y_min

                        # CORRETTO: Filtro bounding box troppo piccole
                        if w >= 5.0 and h >= 5.0:
                            boxes.append([x_min, y_min, x_max, y_max])
                            labels.append(class_id + 1)

        # Resize immagine e box
        image = image.resize((self.input_size, self.input_size))
        scale_x = self.input_size / original_w
        scale_y = self.input_size / original_h

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            boxes[:, [0, 2]] *= scale_x
            boxes[:, [1, 3]] *= scale_y

        target = {"boxes": boxes, "labels": labels}
        image = self.transform(image)

        return image, target

def collate_fn(batch):
    return tuple(zip(*batch))

print("✓ Dataset definito")

In [ ]:
# Cella 4: Preparazione dataset e fold
image_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/images"
label_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/labels"

# Lista file
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(image_files)
print(f"Dataset: {len(image_files)} immagini")

# Crea fold
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)
fold_splits = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(image_files)):
    train_files = [image_files[i] for i in train_idx]
    val_files = [image_files[i] for i in val_idx]
    fold_splits.append({'train': train_files, 'val': val_files})
    print(f"Fold {fold_idx}: {len(train_files)} train, {len(val_files)} val")

print(f"✓ {NUM_FOLDS} fold creati")

In [ ]:
# Cella 5: Funzioni di training e metriche
def train_one_epoch(model, loader, optimizer, device, epoch):
    """Training singola epoca"""
    model.train()
    total_loss = 0
    total_classification_loss = 0
    total_bbox_regression_loss = 0
    
    pbar = tqdm(loader, desc=f"Epoch {epoch+1} Train")
    
    for images, targets in pbar:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        # CORRETTO: Gradient clipping per stabilità
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        optimizer.step()
        
        total_loss += losses.item()
        total_classification_loss += loss_dict.get('classification', torch.tensor(0.0)).item()
        total_bbox_regression_loss += loss_dict.get('bbox_regression', torch.tensor(0.0)).item()
        
        pbar.set_postfix({'loss': f'{losses.item():.3f}'})
    
    n = len(loader)
    return total_loss/n, total_classification_loss/n, total_bbox_regression_loss/n

def validate(model, loader, device):
    """Validazione con loss"""
    model.train()  # Serve per calcolare le loss
    total_loss = 0
    total_classification_loss = 0
    total_bbox_regression_loss = 0
    
    pbar = tqdm(loader, desc="Validation")
    
    with torch.no_grad():
        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            total_loss += losses.item()
            total_classification_loss += loss_dict.get('classification', torch.tensor(0.0)).item()
            total_bbox_regression_loss += loss_dict.get('bbox_regression', torch.tensor(0.0)).item()
            
            pbar.set_postfix({'val_loss': f'{losses.item():.3f}'})
    
    n = len(loader)
    return total_loss/n, total_classification_loss/n, total_bbox_regression_loss/n

def calculate_metrics(model, loader, device, conf_threshold=CONF_THRESHOLD):
    """Calcola mAP e metriche per classe"""
    model.eval()
    metric_map = MeanAveragePrecision(class_metrics=True)
    
    # Contatori per precision/recall per classe
    class_tp = [0, 0, 0]
    class_fp = [0, 0, 0]
    class_fn = [0, 0, 0]
    
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Calculating metrics"):
            images = [img.to(device) for img in images]
            preds = model(images)
            
            # Filtra per confidenza
            filtered_preds = []
            for pred in preds:
                keep = pred['scores'] >= conf_threshold
                filtered_preds.append({
                    'boxes': pred['boxes'][keep].cpu(),
                    'labels': pred['labels'][keep].cpu(),
                    'scores': pred['scores'][keep].cpu()
                })
            
            targets_cpu = [{k: v.cpu() for k, v in t.items()} for t in targets]
            metric_map.update(filtered_preds, targets_cpu)
            
            # Calcola TP/FP/FN per classe
            for pred, target in zip(filtered_preds, targets_cpu):
                pred_boxes = pred['boxes']
                pred_labels = pred['labels']
                gt_boxes = target['boxes']
                gt_labels = target['labels']
                
                if len(pred_boxes) == 0 and len(gt_boxes) == 0:
                    continue
                elif len(pred_boxes) == 0:
                    for lbl in gt_labels:
                        class_fn[lbl.item()-1] += 1
                    continue
                elif len(gt_boxes) == 0:
                    for lbl in pred_labels:
                        class_fp[lbl.item()-1] += 1
                    continue
                
                ious = box_iou(pred_boxes, gt_boxes)
                matched_gt = set()
                
                for pred_idx, pred_label in enumerate(pred_labels):
                    best_iou = 0
                    best_gt_idx = -1
                    
                    for gt_idx, gt_label in enumerate(gt_labels):
                        if gt_idx in matched_gt:
                            continue
                        if pred_label.item() != gt_label.item():
                            continue
                        
                        iou = ious[pred_idx, gt_idx].item()
                        if iou > best_iou:
                            best_iou = iou
                            best_gt_idx = gt_idx
                    
                    cls_idx = pred_label.item() - 1
                    if best_iou >= IOU_THRESHOLD and best_gt_idx != -1:
                        class_tp[cls_idx] += 1
                        matched_gt.add(best_gt_idx)
                    else:
                        class_fp[cls_idx] += 1
                
                for gt_idx, gt_label in enumerate(gt_labels):
                    if gt_idx not in matched_gt:
                        class_fn[gt_label.item()-1] += 1
    
    # Calcola metriche
    map_results = metric_map.compute()
    
    # Metriche per classe
    per_class = {}
    for i, name in enumerate(CLASS_NAMES):
        tp, fp, fn = class_tp[i], class_fp[i], class_fn[i]
        p = tp/(tp+fp) if (tp+fp)>0 else 0.0
        r = tp/(tp+fn) if (tp+fn)>0 else 0.0
        f1 = 2*p*r/(p+r) if (p+r)>0 else 0.0
        per_class[name] = {'precision': p, 'recall': r, 'f1': f1}
    
    # Metriche globali
    total_tp = sum(class_tp)
    total_fp = sum(class_fp)
    total_fn = sum(class_fn)
    
    precision = total_tp/(total_tp+total_fp) if (total_tp+total_fp)>0 else 0.0
    recall = total_tp/(total_tp+total_fn) if (total_tp+total_fn)>0 else 0.0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'mAP50': map_results['map_50'].item(),
        'mAP50-95': map_results['map'].item(),
        'per_class': per_class
    }

print("✓ Funzioni training definite")

In [ ]:
# Cella 6: Training singolo fold
def train_fold(fold_idx, fold_splits, config):
    """Training di un singolo fold"""
    print(f"\n{'='*60}")
    print(f"FOLD {fold_idx}")
    print(f"{'='*60}")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Dataset e loader
    train_dataset = PotholeDataset(image_dir, label_dir, fold_splits[fold_idx]['train'], INPUT_SIZE)
    val_dataset = PotholeDataset(image_dir, label_dir, fold_splits[fold_idx]['val'], INPUT_SIZE)
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], 
                              shuffle=True, collate_fn=collate_fn, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], 
                           shuffle=False, collate_fn=collate_fn, num_workers=2)
    
    # Modello e optimizer
    model = create_model().to(device)
    optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'],
                         momentum=config['momentum'], weight_decay=config['weight_decay'])
    
    # CORRETTO: Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])
    
    # Training loop
    best_map = 0.0
    patience_counter = 0
    history = []
    
    start_time = time.time()
    
    for epoch in range(config['epochs']):
        # Training
        train_loss, train_cls_loss, train_box_loss = train_one_epoch(
            model, train_loader, optimizer, device, epoch
        )
        
        # Validation
        val_loss, val_cls_loss, val_box_loss = validate(model, val_loader, device)
        
        # Learning rate
        lr = optimizer.param_groups[0]['lr']
        scheduler.step()
        
        # Metriche ogni 5 epoche o ultima epoca
        if (epoch + 1) % 5 == 0 or epoch == config['epochs'] - 1:
            metrics = calculate_metrics(model, val_loader, device)
            current_map = metrics['mAP50-95']
        else:
            metrics = None
            current_map = 0.0
        
        # Salva history
        epoch_data = {
            'epoch': epoch + 1,
            'time': time.time() - start_time,
            'train/box_loss': train_box_loss,
            'train/cls_loss': train_cls_loss,
            'train/total_loss': train_loss,
            'val/box_loss': val_box_loss,
            'val/cls_loss': val_cls_loss,
            'val/total_loss': val_loss,
            'lr/pg0': lr,
        }
        
        if metrics:
            epoch_data.update({
                'metrics/precision(B)': metrics['precision'],
                'metrics/recall(B)': metrics['recall'],
                'metrics/mAP50(B)': metrics['mAP50'],
                'metrics/mAP50-95(B)': metrics['mAP50-95'],
            })
        
        history.append(epoch_data)
        
        # Early stopping
        if metrics and current_map > best_map:
            best_map = current_map
            patience_counter = 0
            torch.save(model.state_dict(), f'fold{fold_idx}_best.pth')
            print(f"✓ Epoch {epoch+1}: New best mAP = {best_map:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    training_time = time.time() - start_time
    
    # Carica best model e calcola metriche finali
    model.load_state_dict(torch.load(f'fold{fold_idx}_best.pth'))
    final_metrics = calculate_metrics(model, val_loader, device)
    
    # Salva results.csv
    df = pd.DataFrame(history)
    df.to_csv(f'fold{fold_idx}_results.csv', index=False)
    
    # Prepara dati per .pt
    class_logs = {}
    for i, name in enumerate(CLASS_NAMES):
        class_logs[f'fold{fold_idx}/precision_{name}'] = final_metrics['per_class'][name]['precision']
        class_logs[f'fold{fold_idx}/recall_{name}'] = final_metrics['per_class'][name]['recall']
        class_logs[f'fold{fold_idx}/f1_{name}'] = final_metrics['per_class'][name]['f1']
    
    class_logs.update({
        f'fold{fold_idx}/precision_mean': final_metrics['precision'],
        f'fold{fold_idx}/recall_mean': final_metrics['recall'],
        f'fold{fold_idx}/f1_mean': final_metrics['f1'],
        f'fold{fold_idx}/mAP50_mean': final_metrics['mAP50'],
        f'fold{fold_idx}/mAP_mean': final_metrics['mAP50-95'],
    })
    
    # Salva .pt
    torch.save({
        'fold': fold_idx,
        'class_logs': class_logs,
        'status': 'success',
        'training_time': training_time
    }, f'fold{fold_idx}_results.pt')
    
    print(f"\n✓ Fold {fold_idx} completato:")
    print(f"  Precision: {final_metrics['precision']:.4f}")
    print(f"  Recall: {final_metrics['recall']:.4f}")
    print(f"  mAP@0.5: {final_metrics['mAP50']:.4f}")
    print(f"  mAP@0.5:0.95: {final_metrics['mAP50-95']:.4f}")
    print(f"  Training time: {training_time/60:.1f} min")
    
    return final_metrics, training_time

print("✓ Funzione train_fold definita")

In [ ]:
# Cella 7: Cross-validation completa
def cross_validation():
    """Esegue cross-validation su tutti i fold"""
    print(f"\n🚀 Cross-Validation con {NUM_FOLDS} fold")
    
    # ✅ Inizializza wandb SUBITO all'inizio
    run = wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"Mobilenetv3-ssd320_cv_{NUM_FOLDS}fold",
        config=CONFIG
    )
    run_id = wandb.run.id
    
    all_results = []
    total_start = time.time()
    
    # Disabilita wandb per singoli fold (così non creano run separati)
    os.environ['WANDB_MODE'] = 'disabled'
    
    # Training di tutti i fold
    for fold_idx in range(NUM_FOLDS):
        try:
            metrics, train_time = train_fold(fold_idx, fold_splits, CONFIG)
            all_results.append({
                'fold': fold_idx,
                'precision': metrics['precision'],
                'recall': metrics['recall'],
                'mAP50': metrics['mAP50'],
                'mAP50-95': metrics['mAP50-95'],
                'training_time_min': train_time/60
            })
        except Exception as e:
            print(f"❌ Errore fold {fold_idx}: {e}")
            import traceback
            traceback.print_exc()
    
    # ✅ Riabilita wandb
    if 'WANDB_MODE' in os.environ:
        del os.environ['WANDB_MODE']
    
    total_time = time.time() - total_start
    
    if not all_results:
        print("❌ Nessun fold completato")
        wandb.finish()
        return False
    
    # Statistiche aggregate
    df = pd.DataFrame(all_results)
    summary = {
        'successful_folds': len(all_results),
        'total_time_hours': total_time/3600,
        'precision_mean': df['precision'].mean(),
        'precision_std': df['precision'].std(),
        'recall_mean': df['recall'].mean(),
        'recall_std': df['recall'].std(),
        'mAP50_mean': df['mAP50'].mean(),
        'mAP50_std': df['mAP50'].std(),
        'mAP50-95_mean': df['mAP50-95'].mean(),
        'mAP50-95_std': df['mAP50-95'].std(),
    }
    
    # Salva summary
    df.to_csv('cv_summary.csv', index=False)
    
    print(f"\n{'='*60}")
    print("RISULTATI FINALI")
    print(f"{'='*60}")
    print(f"Fold completati: {len(all_results)}/{NUM_FOLDS}")
    print(f"Tempo totale: {total_time/3600:.2f} ore")
    print(f"\nPrecision: {summary['precision_mean']:.4f} ± {summary['precision_std']:.4f}")
    print(f"Recall: {summary['recall_mean']:.4f} ± {summary['recall_std']:.4f}")
    print(f"mAP@0.5: {summary['mAP50_mean']:.4f} ± {summary['mAP50_std']:.4f}")
    print(f"mAP@0.5:0.95: {summary['mAP50-95_mean']:.4f} ± {summary['mAP50-95_std']:.4f}")
    
    # Log metriche aggregate nello STESSO run
    wandb.log({
        'cv/precision_mean': summary['precision_mean'],
        'cv/recall_mean': summary['recall_mean'],
        'cv/mAP50_mean': summary['mAP50_mean'],
        'cv/mAP50-95_mean': summary['mAP50-95_mean'],
        'experiment/successful_folds': len(all_results),
        'experiment/total_time_hours': total_time/3600
    })
    
    # Log metriche e crea artifact per ogni fold
    for fold_idx in range(NUM_FOLDS):
        pt_path = f'fold{fold_idx}_results.pt'
        csv_path = f'fold{fold_idx}_results.csv'
        pth_path = f'fold{fold_idx}_best.pth'
        
        if os.path.exists(pt_path):
            data = torch.load(pt_path)
            if data.get('status') == 'success':
                wandb.log(data['class_logs'])
                
                # Crea artifact per questo fold
                artifact = wandb.Artifact(f'fold{fold_idx}_results', type='model_results')
                artifact.add_file(pt_path)
                print(f"✓ Fold {fold_idx}: Aggiunto {pt_path}")
                
                if os.path.exists(csv_path):
                    artifact.add_file(csv_path)
                    print(f"✓ Fold {fold_idx}: Aggiunto {csv_path}")
                
                if os.path.exists(pth_path):
                    artifact.add_file(pth_path)
                    print(f"✓ Fold {fold_idx}: Aggiunto {pth_path}")
                
                # Carica artifact del fold
                run.log_artifact(artifact).wait()
                print(f"📦 Fold {fold_idx}: Artifact caricato!")
    
    # Carica summary come artifact separato
    if os.path.exists('cv_summary.csv'):
        summary_artifact = wandb.Artifact('cv_summary', type='summary')
        summary_artifact.add_file('cv_summary.csv')
        run.log_artifact(summary_artifact)
        print("✓ Summary caricato!")
    
    wandb.finish()  # ✅ Chiudi lo STESSO run
    print("\n✅ Tutti gli artifact caricati su wandb!")
    return True

print("✓ Funzione cross_validation definita")

In [ ]:
# Cella 8: Esecuzione training
if __name__ == "__main__":
    success = cross_validation()
    
    if success:
        print("\n🎉 Training completato con successo!")
        print("\n📁 File generati per ogni fold:")
        for fold_idx in range(NUM_FOLDS):
            print(f"  - fold{fold_idx}_best.pth (modello)")
            print(f"  - fold{fold_idx}_results.pt (metriche)")
            print(f"  - fold{fold_idx}_results.csv (training log)")
        print("\n📁 File aggregati:")
        print("  - cv_summary.csv (statistiche cross-validation)")
    else:
        print("\n❌ Training fallito!")

In [ ]:
# Cella 9 (BONUS): Visualizzazione risultati
def visualize_results():
    """Visualizza i risultati del training"""
    import matplotlib.pyplot as plt
    
    # Carica summary
    if not os.path.exists('cv_summary.csv'):
        print("❌ File cv_summary.csv non trovato")
        return
    
    summary = pd.read_csv('cv_summary.csv')
    
    # Plot metriche per fold
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    metrics = ['precision', 'recall', 'mAP50', 'mAP50-95']
    titles = ['Precision', 'Recall', 'mAP@0.5', 'mAP@0.5:0.95']
    
    for ax, metric, title in zip(axes.flat, metrics, titles):
        values = summary[metric].values
        folds = summary['fold'].values
        
        ax.bar(folds, values, color='steelblue', alpha=0.7)
        ax.axhline(values.mean(), color='red', linestyle='--', 
                   label=f'Mean: {values.mean():.4f}')
        ax.set_xlabel('Fold')
        ax.set_ylabel(title)
        ax.set_title(f'{title} per Fold')
        ax.legend()
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('cv_metrics.png', dpi=150)
    plt.show()
    
    print("\n✓ Grafico salvato come cv_metrics.png")
    
    # Plot training curves per un fold
    fold_to_plot = 0
    csv_path = f'fold{fold_to_plot}_results.csv'
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss
        ax = axes[0]
        if 'train/total_loss' in df.columns:
            ax.plot(df['epoch'], df['train/total_loss'], label='Train Loss', marker='o')
        if 'val/total_loss' in df.columns:
            ax.plot(df['epoch'], df['val/total_loss'], label='Val Loss', marker='s')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.set_title(f'Fold {fold_to_plot} - Training & Validation Loss')
        ax.legend()
        ax.grid(alpha=0.3)
        
        # Metriche
        ax = axes[1]
        metric_cols = [col for col in df.columns if col.startswith('metrics/')]
        for col in metric_cols:
            values = df[col].dropna()
            if len(values) > 0:
                epochs = df.loc[values.index, 'epoch']
                label = col.replace('metrics/', '').replace('(B)', '')
                ax.plot(epochs, values, label=label, marker='o')
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Metric Value')
        ax.set_title(f'Fold {fold_to_plot} - Metrics')
        ax.legend()
        ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'fold{fold_to_plot}_curves.png', dpi=150)
        plt.show()
        
        print(f"✓ Grafico fold {fold_to_plot} salvato come fold{fold_to_plot}_curves.png")

# Visualizza i risultati (opzionale, decommentare per usare)
visualize_results()

print("✓ Funzione visualize_results definita (decommentare per usare)")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_predictions(model, val_dataset, device, num_images=20, score_threshold=0.5):
    """
    Esegue e visualizza le predizioni del modello su immagini di validazione in una griglia 2xN.
    
    Args:
        model: Modello SSD PyTorch già caricato con i pesi.
        val_dataset: Oggetto Dataset per la validazione (istanza di PotholeDataset).
        device: torch.device (cuda o cpu).
        num_images: Numero di immagini da predire e visualizzare.
        score_threshold: Soglia minima per visualizzare una predizione.
    """
    model.eval()
    model.to(device)

    class_names = ["background"] + CLASS_NAMES
    colors = ["gray", "red", "orange", "blue"]

    indices = random.sample(range(len(val_dataset)), num_images)
    n_cols = 2
    n_rows = math.ceil(num_images / n_cols)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(14, 6 * n_rows))
    axs = axs.flatten()

    for plot_idx, dataset_idx in enumerate(indices):
        image_tensor, _ = val_dataset[dataset_idx]
        image = image_tensor.unsqueeze(0).to(device)

        img_np = image_tensor.permute(1, 2, 0).numpy()

        with torch.no_grad():
            prediction = model(image)[0]

        ax = axs[plot_idx]
        ax.imshow(img_np)

        for box, label, score in zip(prediction["boxes"], prediction["labels"], prediction["scores"]):
            if score < score_threshold:
                continue

            box = box.cpu().numpy()
            label = label.item()
            score = score.item()

            x_min, y_min, x_max, y_max = box
            width = x_max - x_min
            height = y_max - y_min

            rect = patches.Rectangle(
                (x_min, y_min), width, height,
                linewidth=2, edgecolor=colors[label], facecolor='none'
            )
            ax.add_patch(rect)
            ax.text(
                x_min, y_min - 5,
                f"{class_names[label]}: {score:.2f}",
                color=colors[label], fontsize=10,
                backgroundcolor='black'
            )

        ax.axis("off")
        ax.set_title(f"Img: {val_dataset.image_files[dataset_idx]}", fontsize=10)

    # Disabilita eventuali assi non usati
    for i in range(plot_idx + 1, len(axs)):
        axs[i].axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fold = 0  # o qualsiasi fold disponibile

# Carica modello e pesi
model = create_model(num_classes=4, size=320)
model.load_state_dict(torch.load(f"fold{fold}_best.pth"))

# Dataset di validazione
val_dataset = PotholeDataset(
    image_dir, label_dir,
    fold_splits[fold]['val'],
    input_size=320
)

# Visualizza le predizioni
visualize_predictions(model, val_dataset, device, num_images=40, score_threshold=0.5)